## Python notebook 
V nasledujúcom notebooku sú popísané Python metódy použité pre predspracovanie dát z anotačného projektu. Obsah notebooku je nasledujúci:
1. Import potrebných knižníc
2. Predspracovanie csv súboru anotačného projektu
3. Extrakcia udalostí
4. Rozdelenie zložiek na testovaciu a trénovaciu časť

# 1. Import potrebných knižníc

In [ ]:
import pandas as pd
import csv
import os
import cv2
import numpy as np
import operator
from skimage.io import imread
from PIL import Image, ImageDraw
import json
import requests
import splitfolders

# 2. Predspracovanie csv súboru anotačného projektu
Csv súbor anotačného projektu sme upravili do vhodnej podoby pre ďalšiu prácu a rozdelili ho na menšie csv súbory obsahujúce len udalosti rovnakej triedy podľa počtu disperzií.

In [3]:
#csv anotacného projektu
data=pd.read_csv("jjmarkers-classifications2.csv")

In [4]:
#cesta na uloženie rozparsovaného csv
csvOut='/home/jovyan/data/lightning/Samuel/2_roc_zs/vyrezavanie_tweekov/annot_output.csv'

In [5]:
data=data.drop(data.index[0:50])

In [6]:
a=50

In [7]:
#rozparsovanie
with open(csvOut, mode='w') as csv_file:
    fieldnames = ['image','xmin', 'ymin', 'xmax', 'ymax', 'label', 'tweek_sign']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    while (a<=len(data)-1):
        load=json.loads(data.iloc[a, 11])[0]["value"]
        filesy=json.loads(data.iloc[a,12])[str(data.iloc[a,13])]["Filename"].split('.')
        filesy=filesy[0]+'.jpg'
        for k in load:
            writer.writerow({'image': filesy,'xmin': round(k["x"]),'ymin': 0,'xmax': round(k["x"]+k["width"]),  
            'ymax': 162,'label': k["tool_label"], 'tweek_sign': k["details"]})
        a+=1

In [8]:
#uprava datasetu

In [2]:
data2 = pd.read_csv("annot_output.csv")

In [3]:
data2["tweek_sign"].replace({"[]": 0, "[{'value': '1'}]": 1, "[{'value': 1}]": 1, "[{'value': 2}]": 2, "[{'value': 3}]": 3, "[{'value': 4}]": 4, "[{'value': 5}]": 5, "[{'value': 6}]": 6, "[{'value': 7}]": 7, "[{'value': 8}]": 8,"[{'value': 9}]": 9}, inplace=True)

In [4]:
data2['tweek_sign'].unique()

array([0, 2, 1, 5, 4, 6, 3, 9, 7, 8])

In [5]:
data2['label'].replace({"Blesk": 0, "Blesk s chvostíkom": 1}, inplace=True)

In [6]:
data2['label'].unique()

array([0, 1])

In [14]:
data2.to_csv('anot_data_all.csv', index=False)

In [15]:
print(data2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22183 entries, 0 to 22182
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image       22183 non-null  object
 1   xmin        22183 non-null  int64 
 2   ymin        22183 non-null  int64 
 3   xmax        22183 non-null  int64 
 4   ymax        22183 non-null  int64 
 5   label       22183 non-null  int64 
 6   tweek_sign  22183 non-null  int64 
dtypes: int64(6), object(1)
memory usage: 1.2+ MB
None


In [16]:
#iba tweeky

In [17]:
data3 = data2[(data2['label'] == 1)]

In [18]:
data3.to_csv('anot_data_tweek.csv', index=False)

In [ ]:
#rozdelenie podľa počtu disperzií

In [7]:
tweekSet = pd.read_csv("anot_data_tweek.csv")

In [21]:
tweek1 = tweekSet[(tweekSet['tweek_sign'] == 1)]
tweek1.to_csv('tweek1.csv', index=False)

In [22]:
tweek2 = tweekSet[(tweekSet['tweek_sign'] == 2)]
tweek2.to_csv('tweek2.csv', index=False)

In [23]:
tweek3 = tweekSet[(tweekSet['tweek_sign'] == 3)]
tweek3.to_csv('tweek3.csv', index=False)

In [10]:
tweekMoreThan4 = tweekSet[(tweekSet['tweek_sign'] >= 4)]
tweekMoreThan4.to_csv('tweekMoreThan4.csv', index=False)

# 3. Extrakcia udalostí
Z príslušného csv sme na základe koodrinátov extrahovali udalosti a uložili ich do samostatných zložiek podľa triedy počtu disperzií.

In [2]:
# imagePath - cesta ku spektrogramom
imagePath ='images'

In [ ]:
#tweeky s jednou disperziou

In [3]:
#savePath - zložka, do ktorej sa uložia extrahované udalosti
savePath = 'data/tweek1'

In [4]:
#extrakcia udalostí zo spektrogramov
with open('tweek1.csv', 'r') as read_obj:
    csv_reader = csv.reader(read_obj)
    header = next(csv_reader)
    if header != None:
        for row in csv_reader:
            try:
                img = Image.open(os.path.join(imagePath, row[0]))
                im = img.crop((int(row[1]), 0, int(row[3]), 162)).save(os.path.join(savePath, row[0].split(".")[0]+"_%d.jpg"%eventNR))
                #print(row[0] + " cropped")
                eventNR += 1
            except:
                continue   

In [ ]:
#tweeky s 2 disperziami

In [5]:
savePath = 'data/tweek2'

In [6]:
with open('tweek2.csv', 'r') as read_obj:
    csv_reader = csv.reader(read_obj)
    header = next(csv_reader)
    if header != None:
        for row in csv_reader:
            try:
                img = Image.open(os.path.join(imagePath, row[0]))
                im = img.crop((int(row[1]), 0, int(row[3]), 162)).save(os.path.join(savePath, row[0].split(".")[0]+"_%d.jpg"%eventNR))
                #print(row[0] + " cropped")
                eventNR += 1
            except:
                continue   

In [ ]:
#tweeky s 3 disperziami

In [7]:
savePath = 'data/tweek3'

In [8]:
with open('tweek3.csv', 'r') as read_obj:
    csv_reader = csv.reader(read_obj)
    header = next(csv_reader)
    if header != None:
        for row in csv_reader:
            try:
                img = Image.open(os.path.join(imagePath, row[0]))
                im = img.crop((int(row[1]), 0, int(row[3]), 162)).save(os.path.join(savePath, row[0].split(".")[0]+"_%d.jpg"%eventNR))
                #print(row[0] + " cropped")
                eventNR += 1
            except:
                continue   

In [ ]:
#tweeky so 4 a viac disperziami

In [10]:
savePath = 'data/tweek4more'

In [11]:
with open('tweekMoreThan4.csv', 'r') as read_obj:
    csv_reader = csv.reader(read_obj)
    header = next(csv_reader)
    if header != None:
        for row in csv_reader:
            try:
                img = Image.open(os.path.join(imagePath, row[0]))
                im = img.crop((int(row[1]), 0, int(row[3]), 162)).save(os.path.join(savePath, row[0].split(".")[0]+"_%d.jpg"%eventNR))
                print(row[0] + " cropped")
                eventNR += 1
            except:
                continue   

ch1_20150902_230432_765_s38.jpg cropped
ch1_20150902_230432_765_s38.jpg cropped
ch1_20150902_232933_901_s85.jpg cropped
ch1_20150902_232933_901_s68.jpg cropped
ch1_20180602_203728_547_s22.jpg cropped
ch1_20150902_231933_448_s115.jpg cropped
ch1_20150902_231933_448_s115.jpg cropped
ch1_20150902_231933_448_s115.jpg cropped
ch1_20150902_231933_448_s115.jpg cropped
ch1_20150902_230432_765_s52.jpg cropped
ch1_20150902_230432_765_s52.jpg cropped
ch1_20150902_230432_765_s52.jpg cropped
ch1_20160830_194001_403_s70.jpg cropped
ch1_20180615_202006_477_s110.jpg cropped
ch1_20150902_230932_991_s81.jpg cropped
ch1_20180615_200505_862_s56.jpg cropped
ch1_20160830_191500_286_s90.jpg cropped
ch1_20180602_211229_947_s144.jpg cropped
ch1_20160830_192500_741_s60.jpg cropped
ch1_20160830_192500_741_s60.jpg cropped
ch1_20180615_200005_654_s40.jpg cropped
ch1_20180615_202006_477_s55.jpg cropped
ch1_20180602_212230_349_s89.jpg cropped
ch1_20180602_205229_138_s103.jpg cropped
ch1_20150902_235935_260_s70.jpg c

# 4. Rozdelenie zložiek na testovaniu a trénovaciu časť

Zložky z extrahovanými udalosťami sme rozdelili v pomere 90% trénovanie a 10% testovanie.

In [ ]:
#rozdelenie na trénovaciu a testovaciu časť
splitfolders.ratio(input_folder, output = "data_split_smoothed", seed = 42, ratio = (.9, .1))